<a href="https://colab.research.google.com/github/srimancho9/Week-4-Retrieval-Augmented-Generation-RAG-/blob/main/Week4_RAG_LangChain_AppliedGenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔎 Hands-On: Retrieval-Augmented Generation (RAG) with LangChain + Chroma

**Last updated:** 2025-09-08 23:15

**Why this topic?**
- Bridges the gap between *playing with LLMs* and building **end-to-end applications**.
- Introduces **retrieval pipelines, embeddings, and vector databases**.
- **LangChain (or LlamaIndex)** orchestrates these components in a reproducible workflow.

**Agenda (45–60 min)**
1. Intro: Why RAG? Reducing hallucinations by grounding in data
2. Install & Setup
3. Load & Chunk Documents
4. Store/Retrieve with Chroma
5. Connect an LLM (HF or OpenAI)
6. Pipeline test (ask Qs about docs)
7. Mini-experiments: embedding swap, chunk size sensitivity
8. Log reproducibility
9. Wrap-up tasks

In [2]:
# Install dependencies
!pip -q install -U langchain langchain-community chromadb sentence-transformers pypdf transformers accelerate
# Optional OpenAI
# %pip -q install -U openai tiktoken langchain-openai

In [3]:
import json, sys, platform, os, chromadb, transformers, sentence_transformers
try:
    import torch
    torch_v = torch.__version__
    cuda_ok = torch.cuda.is_available()
    device_name = torch.cuda.get_device_name(0) if cuda_ok else "CPU"
except:
    torch_v, cuda_ok, device_name = "N/A", False, "CPU"

env = {
    "python": sys.version,
    "platform": platform.platform(),
    "torch": torch_v,
    "cuda": cuda_ok,
    "device": device_name,
    "transformers": transformers.__version__,
    "sentence_transformers": sentence_transformers.__version__,
    "chromadb": chromadb.__version__
}
print(json.dumps(env, indent=2))
with open("env_rag.json","w") as f: json.dump(env, f, indent=2)

{
  "python": "3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]",
  "platform": "Linux-6.1.123+-x86_64-with-glibc2.35",
  "torch": "2.8.0+cu126",
  "cuda": false,
  "device": "CPU",
  "transformers": "4.56.1",
  "sentence_transformers": "5.1.0",
  "chromadb": "1.1.0"
}


In [4]:
sample_text = """
Retrieval-Augmented Generation (RAG) is a hybrid approach that combines the strengths of large language models (LLMs) with external knowledge retrieval. Instead of relying solely on pre-trained parameters, RAG dynamically fetches relevant information from a knowledge base—such as a vector database, document store, or API—before generating a response. This improves factual accuracy, reduces hallucinations, and makes systems more adaptable to evolving domains. A common pipeline includes document chunking, embedding with models like Sentence-BERT, storing vectors in databases such as FAISS or Pinecone, and using similarity search during query time. Applications of RAG include customer support bots, legal document analysis, personalized tutoring systems, and enterprise knowledge assistants.
"""
with open("sample.txt","w", encoding="utf-8") as f: f.write(sample_text)
print("Created sample.txt")

Created sample.txt


In [5]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

docs = TextLoader("sample.txt", encoding="utf-8").load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print("Chunks:", len(chunks))
print("First chunk:\n", chunks[0].page_content[:300])

Chunks: 2
First chunk:
 Retrieval-Augmented Generation (RAG) is a hybrid approach that combines the strengths of large language models (LLMs) with external knowledge retrieval. Instead of relying solely on pre-trained parameters, RAG dynamically fetches relevant information from a knowledge base—such as a vector database, 


In [6]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

emb = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(chunks, emb, persist_directory="chroma_minilm")
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
print("Chroma DB ready")

/tmp/ipython-input-2748661336.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chroma DB ready


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # fallback: "distilgpt2"
tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
pipe = pipeline("text-generation", model=model, tokenizer=tok, max_new_tokens=200)
llm = HuggingFacePipeline(pipeline=pipe)
print("LLM ready:", MODEL_ID)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


LLM ready: TinyLlama/TinyLlama-1.1B-Chat-v1.0


/tmp/ipython-input-2989922931.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
q = "What Type of storing  vectors database is included in RAG?"
print("Q:", q)
print("A:", qa.run(q))

Q: What Type of storing  vectors database is included in RAG?
A: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

and makes systems more adaptable to evolving domains. A common pipeline includes document chunking, embedding with models like Sentence-BERT, storing vectors in databases such as FAISS or Pinecone, and using similarity search during query time. Applications of RAG include customer support bots, legal document analysis, personalized tutoring systems, and enterprise knowledge assistants.

Retrieval-Augmented Generation (RAG) is a hybrid approach that combines the strengths of large language models (LLMs) with external knowledge retrieval. Instead of relying solely on pre-trained parameters, RAG dynamically fetches relevant information from a knowledge base—such as a vector database, document store, or API—before generating a response. This improves factual accura

In [10]:
emb_e5 = SentenceTransformerEmbeddings(model_name="intfloat/e5-small-v2")
vectordb_e5 = Chroma.from_documents(chunks, emb_e5, persist_directory="chroma_e5")
qa_e5 = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb_e5.as_retriever(), chain_type="stuff")
print("MiniLM vs E5-small test:\n")
print("MiniLM:", qa.run("List two GenAI techniques emphasized."))
print("E5-small:", qa_e5.run("List two GenAI techniques emphasized."))

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

MiniLM vs E5-small test:

MiniLM: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Retrieval-Augmented Generation (RAG) is a hybrid approach that combines the strengths of large language models (LLMs) with external knowledge retrieval. Instead of relying solely on pre-trained parameters, RAG dynamically fetches relevant information from a knowledge base—such as a vector database, document store, or API—before generating a response. This improves factual accuracy, reduces hallucinations, and makes systems more adaptable to evolving domains. A common pipeline includes document

and makes systems more adaptable to evolving domains. A common pipeline includes document chunking, embedding with models like Sentence-BERT, storing vectors in databases such as FAISS or Pinecone, and using similarity search during query time. Applications of RAG include customer support bots, legal 

In [11]:
splitter_small = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks_small = splitter_small.split_documents(docs)
vectordb_small = Chroma.from_documents(chunks_small, emb)
qa_small = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb_small.as_retriever(), chain_type="stuff")
print("Default chunks:", qa.run("Summarize the RAG in one sentence."))
print("Smaller chunks:", qa_small.run("Summarize the RAG in one sentence."))

Default chunks: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Retrieval-Augmented Generation (RAG) is a hybrid approach that combines the strengths of large language models (LLMs) with external knowledge retrieval. Instead of relying solely on pre-trained parameters, RAG dynamically fetches relevant information from a knowledge base—such as a vector database, document store, or API—before generating a response. This improves factual accuracy, reduces hallucinations, and makes systems more adaptable to evolving domains. A common pipeline includes document

and makes systems more adaptable to evolving domains. A common pipeline includes document chunking, embedding with models like Sentence-BERT, storing vectors in databases such as FAISS or Pinecone, and using similarity search during query time. Applications of RAG include customer support bots, legal document analysis,

In [12]:
repro = {
    "embedding_models": ["all-MiniLM-L6-v2","intfloat/e5-small-v2"],
    "chunking": [{"size":500,"overlap":100},{"size":300,"overlap":50}],
    "llm": MODEL_ID
}
with open("rag_run_config.json","w") as f: json.dump(repro,f,indent=2)
print("Saved rag_run_config.json")

Saved rag_run_config.json
